In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
import numpy as np
from annoy import AnnoyIndex
import pickle

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#checkpoint_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  

In [3]:
checkpoint_path = "/home/uddeshya.singh/DataSet_triplet/Roberta_classification_r"  

In [4]:
def load_finetuned_model(checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    model = model.to(device)
    print("Loading done")
    return model, tokenizer, device

model, tokenizer, device = load_finetuned_model(checkpoint_path)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/DataSet_triplet/Roberta_classification_r and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading done


In [5]:
# input_pickle_path = 'DataSet_triplet/PUNJAB_5_for_lookup_without_city.pkl'
input_pickle_path = 'DataSet_triplet/classification_dataset/PUNJAB_2_for_lookup_without_city.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

print(f"Length of addresses: {len(addresses)}, {addresses[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs)}, {lat_lngs[-1]}")

Length of addresses: 3642886, wazidowal mj fectory wazidowal
Length of lat_lngs: 3642886, [31.229235499999998, 75.76919190000002]


In [6]:
def get_batch_embeddings(model, tokenizer, device, texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [7]:
def compute_embeddings_in_batches(model, tokenizer, device, addresses, batch_size=128):
    all_embeddings = []
    for i in tqdm(range(0, len(addresses), batch_size), desc="Computing embeddings"):
        batch_texts = addresses[i:i + batch_size]
        batch_embeddings = get_batch_embeddings(model, tokenizer, device, batch_texts)
        all_embeddings.append(batch_embeddings)
    return np.vstack(all_embeddings)

In [8]:
embeddings = compute_embeddings_in_batches(model, tokenizer, device, addresses)

Computing embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28461/28461 [55:08<00:00,  8.60it/s]


### take 2 hrs to generate embedding for full address(more than 1)

In [9]:
embeddings_save_path = 'DataSet_triplet/embeddings_ANN_2_without_city_Roberta_classification_R.pkl'

with open(embeddings_save_path, 'wb') as f:
    pickle.dump(embeddings, f)

print(f"Embeddings saved to {embeddings_save_path}")


Embeddings saved to DataSet_triplet/embeddings_ANN_2_without_city_Roberta_classification_R.pkl


In [12]:
with open(embeddings_save_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

Loaded embeddings shape: (2249853, 768)


In [10]:
with open(embeddings_save_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

Loaded embeddings shape: (3642886, 768)
